Import the necessary libraries

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point, Polygon, MultiLineString
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.ops import unary_union, polygonize
from scipy.spatial import Delaunay
#import alphashape
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy.interpolate import splprep, splev
import folium
from shapely.geometry import Point, Polygon
from census import Census
import requests
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
from rasterio.features import shapes
from rasterio.warp import calculate_default_transform, reproject, Resampling


In [5]:
#importing Texas County and District shapefiles
shapefile_path1 = "Texas_County_Boundaries_953635247820289118/County.shp"
county = gpd.read_file(shapefile_path1)
shapefile_path2 = "TxDOT_Districts_5167211860954174164/TxDOT_Districts.shp"
district = gpd.read_file(shapefile_path2)

In [6]:
#Importing monitored roads shapefile
shapefile_path4 = "2023 Monitored Roads Dissolved/new_monitored/newmonitored.shp"
monitored_roads = gpd.read_file(shapefile_path4)

In [4]:
monitored_roads.head()

,TOP100ID,Shape_Leng,geometry
0,20010,0.070503,"LINESTRING (-97.77001 30.24843, -97.76954 30.2..."
1,20021,0.061771,"LINESTRING (-97.76391 30.22296, -97.76397 30.2..."
2,20040,0.043798,"LINESTRING (-97.74749 30.26423, -97.74769 30.2..."
3,20041,0.063094,"LINESTRING (-97.76941 30.22661, -97.76970 30.2..."
4,200370,0.076132,"LINESTRING (-101.95458 34.98532, -101.95302 34..."


In [60]:
county.tail()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,county_geometry,state,county,county_area
249,148,148,48295,148,4,Lipscomb,249,"POLYGON ((-11187636.512 4369621.983, -11187008...",48,295,3.726065e+09
250,179,179,48357,179,4,Ochiltree,250,"POLYGON ((-11228849.224 4369635.829, -11227802...",48,357,3.671226e+09
251,98,98,48195,99,4,Hansford,251,"POLYGON ((-11252750.637 4369621.979, -11252747...",48,195,3.676774e+09
252,119,119,48237,120,2,Jack,198,"POLYGON ((-10900201.697 3953021.671, -10900201...",48,237,3.416516e+09
253,64,64,48127,64,22,Dimmit,17,"POLYGON ((-11144652.652 3330733.031, -11144652...",48,127,4.431715e+09


In [9]:
#Get the state and county numbers from FIPS
# Convert the values to strings and split after '48'
county['state'] = county['FIPS_ST_CN'].astype(str).str[:2]  # First two digits ('48')
county['county'] = county['FIPS_ST_CN'].astype(str).str[2:]  # Remaining digits

In [10]:
#importing MPO shapefile
shapefile_path3 = "mpo_texas/Metropolitan_Planning_Organization.shp"
mpo = gpd.read_file(shapefile_path3)

In [11]:
district.head()

,TXDOT_DIST,TXDOT_DI_1,TXDOT_DI_2,DIST_ABRVN,DIST_NBR,DIST_NM,TYPE,geometry
0,ABL,8,Abilene,ABL,8,Abilene,Rural,"POLYGON ((-11245849.009 3948145.324, -11245233..."
1,AMA,4,Amarillo,AMA,4,Amarillo,Rural,"POLYGON ((-11187636.512 4369621.983, -11187008..."
2,ATL,19,Atlanta,ATL,19,Atlanta,Rural,"POLYGON ((-10542321.690 3989178.361, -10542278..."
3,AUS,14,Austin,AUS,14,Austin,Metro,"POLYGON ((-10899469.912 3637296.602, -10899470..."
4,BMT,20,Beaumont,BMT,20,Beaumont,Urban,"POLYGON ((-10413244.465 3656589.521, -10413223..."


In [12]:
county.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,geometry,state,county
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325


In [13]:
county.shape

(254, 10)

In [14]:
county["TXDOT_DIST"].unique().shape

(25,)

In [15]:
# Merge the county and district shapefiles
merged_count_dist = pd.merge(county, district, left_on='TXDOT_DIST', right_on='DIST_NBR', how='left')
merged_count_dist.shape

(254, 18)

In [16]:
merged_count_dist.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST_x,CNTY_NM,GID,geometry_x,state,county,TXDOT_DIST_y,TXDOT_DI_1,TXDOT_DI_2,DIST_ABRVN,DIST_NBR,DIST_NM,TYPE,geometry_y
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,SAT,15,San Antonio,Metro,"POLYGON ((-11103341.987 3540952.941, -11103526..."
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,YKM,13,Yoakum,YKM,13,Yoakum,Rural,"POLYGON ((-10758694.531 3522214.151, -10758867..."
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,HOU,12,Houston,HOU,12,Houston,Metro,"POLYGON ((-10642059.507 3569163.765, -10642010..."
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,YKM,13,Yoakum,YKM,13,Yoakum,Rural,"POLYGON ((-10758694.531 3522214.151, -10758867..."
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,SAT,15,San Antonio,SAT,15,San Antonio,Metro,"POLYGON ((-11103341.987 3540952.941, -11103526..."


In [17]:
mpo.head()

,GID,MPO_NM,MPO_LBL,MPO_NBR,geometry
0,33.0,Lubbock,Lubbock MPO,122.0,"POLYGON ((-11359524.986 3972189.055, -11361181..."
1,51.0,Wichita Falls,Wichita Falls MPO,151.0,"POLYGON ((-10980354.224 4015027.775, -10979508..."
2,52.0,Permian Basin,Permian Basin MPO,174.0,"POLYGON ((-11397766.658 3727864.480, -11399086..."
3,53.0,Rio Grande Valley,Rio Grande Valley MPO,230.0,"POLYGON ((-10974318.889 3029480.644, -10974307..."
4,34.0,Houston-Galveston Area Council,Houston-Galveston Area Council MPO,15.0,"POLYGON ((-10642059.508 3569163.765, -10642010..."


In [18]:
#merge MPO shapefile
merged_count_dist_mpo = pd.merge(merged_count_dist, mpo, left_on='GID', right_on='GID', how='left')
merged_count_dist_mpo.shape

(254, 22)

In [19]:
merged_count_dist_mpo.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST_x,CNTY_NM,GID,geometry_x,state,county,...,TXDOT_DI_2,DIST_ABRVN,DIST_NBR,DIST_NM,TYPE,geometry_y,MPO_NM,MPO_LBL,MPO_NBR,geometry
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,...,San Antonio,SAT,15,San Antonio,Metro,"POLYGON ((-11103341.987 3540952.941, -11103526...",South East Texas Regional Planning Commission,South East Texas RPC MPO,135.0,"POLYGON ((-10472027.847 3571354.051, -10472022..."
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,...,Yoakum,YKM,13,Yoakum,Rural,"POLYGON ((-10758694.531 3522214.151, -10758867...",Victoria,Victoria MPO,363.0,"POLYGON ((-10792171.645 3384911.927, -10791169..."
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,...,Houston,HOU,12,Houston,Metro,"POLYGON ((-10642059.507 3569163.765, -10642010...",Bryan-College Station,Bryan-College Station MPO,249.0,"POLYGON ((-10713061.325 3627506.843, -10713050..."
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,...,Yoakum,YKM,13,Yoakum,Rural,"POLYGON ((-10758694.531 3522214.151, -10758867...",Texarkana,Texarkana MPO,211.0,"POLYGON ((-10486938.781 3949087.997, -10487007..."
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,...,San Antonio,SAT,15,San Antonio,Metro,"POLYGON ((-11103341.987 3540952.941, -11103526...",Waco,Waco MPO,140.0,"POLYGON ((-10833157.277 3666435.715, -10836210..."


In [16]:
merged_count_dist_mpo["GID"].unique()

array([ 35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
        48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
       100, 101, 102,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
        11,  12,  13,  14,  15,  16,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 178, 179, 180, 181, 182, 183, 18

In [17]:
merged_count_dist_mpo.shape

(254, 22)

In [18]:
merged_count_dist_mpo.columns

Index(['CMPTRL_CNT', 'DPS_CNTY_N', 'FIPS_ST_CN', 'TXDOT_CNTY', 'TXDOT_DIST_x',
       'CNTY_NM', 'GID', 'geometry_x', 'state', 'county', 'TXDOT_DIST_y',
       'TXDOT_DI_1', 'TXDOT_DI_2', 'DIST_ABRVN', 'DIST_NBR', 'DIST_NM', 'TYPE',
       'geometry_y', 'MPO_NM', 'MPO_LBL', 'MPO_NBR', 'geometry'],
      dtype='object')

In [20]:
merged_count_dist_mpo_1 = merged_count_dist_mpo[['FIPS_ST_CN', 'TXDOT_CNTY','CNTY_NM','geometry_x', 
                                                 'state', 'county','DIST_ABRVN', 'DIST_NBR','DIST_NM',
                                                 'geometry_y', 'MPO_NM', 'MPO_LBL', 'MPO_NBR', 'geometry']]
merged_count_dist_mpo_1.head()

,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,MPO_NM,MPO_LBL,MPO_NBR,geometry
0,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",South East Texas Regional Planning Commission,South East Texas RPC MPO,135.0,"POLYGON ((-10472027.847 3571354.051, -10472022..."
1,48285,143,Lavaca,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Victoria,Victoria MPO,363.0,"POLYGON ((-10792171.645 3384911.927, -10791169..."
2,48039,20,Brazoria,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,HOU,12,Houston,"POLYGON ((-10642059.507 3569163.765, -10642010...",Bryan-College Station,Bryan-College Station MPO,249.0,"POLYGON ((-10713061.325 3627506.843, -10713050..."
3,48481,241,Wharton,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Texarkana,Texarkana MPO,211.0,"POLYGON ((-10486938.781 3949087.997, -10487007..."
4,48325,163,Medina,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",Waco,Waco MPO,140.0,"POLYGON ((-10833157.277 3666435.715, -10836210..."


Population Data only 1 year

In [20]:
# Insert your API key here
API_KEY = "2d82444d2135e89607b5c75cef00565222fac1a9"
c = Census(API_KEY)

population_data = c.acs5.state_county(['B01003_001E', 'B03003_002E', 'B03002_003E', 'B03002_004E', 'B03003_003E', 'B03002_013E','B03002_014E','B02009_001E', 'B02010_001E','B02011_001E', 'B02012_001E','B02013_001E'], '48', Census.ALL, 
    year=2021)

# Convert to a DataFrame for easier analysis
population_df = pd.DataFrame(population_data)

# Rename columns to more descriptive names
population_df = population_df.rename(columns={
    'B01003_001E': 'Total_Population',
    'B03003_002E': 'Not_Hispanic_or_Latino',
    'B03002_003E': 'White_Alone',
    'B03002_004E': 'Black_or_African_American_Alone',
    'B03003_003E': 'Hispanic_or_Latino',
    'B03002_013E': 'Asian_Alone',
    'B03002_014E': 'Native_Hawaiian_or_Other_Pacific_Islander_Alone',
    'B02009_001E': 'Native_American_Alone',
    'B02010_001E': 'Other_Race',
    'B02011_001E': 'Two_or_More_Races',
    'B02012_001E': 'Some_Other_Race',
    'B02013_001E': 'Other_Race_Two_or_More'
})


#5 YEAR ACS Population data by multiple year by county

In [21]:
# Initialize an empty list to store DataFrames for each year
population_dfs = []

# List of years you want to retrieve data for
years = [2019, 2020, 2021, 2022]

# Loop through each year to retrieve data and store it with a 'Year' column
for year in years:
    population_data = c.acs5.state_county(
        ['B01003_001E', 'B03003_002E', 'B03002_003E', 'B03002_004E', 'B03003_003E', 
         'B03002_013E', 'B03002_014E', 'B02009_001E', 'B02010_001E', 'B02011_001E', 
         'B02012_001E', 'B02013_001E'], 
        '48', Census.ALL, 
        year=year
    )
    
    # Convert to DataFrame and add a column for the year
    population_df = pd.DataFrame(population_data)
    population_df['Year'] = year  # Add year column for easy tracking
    
    # Rename columns for readability
    population_df = population_df.rename(columns={
        'B01003_001E': 'Total_Population',
        'B03003_002E': 'Not_Hispanic_or_Latino',
        'B03002_003E': 'White_Alone',
        'B03002_004E': 'Black_or_African_American_Alone',
        'B03003_003E': 'Hispanic_or_Latino',
        'B03002_013E': 'Asian_Alone',
        'B03002_014E': 'Native_Hawaiian_or_Other_Pacific_Islander_Alone',
        'B02009_001E': 'Native_American_Alone',
        'B02010_001E': 'Other_Race',
        'B02011_001E': 'Two_or_More_Races',
        'B02012_001E': 'Some_Other_Race',
        'B02013_001E': 'Other_Race_Two_or_More'
    })
    
    # Append to the list of DataFrames
    population_dfs.append(population_df)

# Concatenate all yearly DataFrames into a single DataFrame
all_years_population_df = pd.concat(population_dfs, ignore_index=True)



In [22]:
all_years_population_df.head()

,Total_Population,Not_Hispanic_or_Latino,White_Alone,Black_or_African_American_Alone,Hispanic_or_Latino,Asian_Alone,Native_Hawaiian_or_Other_Pacific_Islander_Alone,Native_American_Alone,Other_Race,Two_or_More_Races,Some_Other_Race,Other_Race_Two_or_More,state,county,Year
0,28180.0,24487.0,21043.0,2517.0,3693.0,3091.0,81.0,3040.0,364.0,34.0,91.0,535.0,48,407,2019
1,41018.0,37494.0,32971.0,3337.0,3524.0,2558.0,33.0,3633.0,588.0,243.0,0.0,1071.0,48,459,2019
2,51832.0,36251.0,22222.0,12831.0,15581.0,7874.0,80.0,13165.0,783.0,635.0,31.0,7288.0,48,473,2019
3,49173.0,29660.0,28045.0,600.0,19513.0,17503.0,90.0,959.0,1266.0,377.0,58.0,1072.0,48,493,2019
4,23064.0,12041.0,10824.0,738.0,11023.0,9559.0,8.0,934.0,178.0,240.0,45.0,1353.0,48,219,2019


In [32]:
population_df["county"].unique()

array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
       '019', '021', '023', '025', '027', '029', '031', '033', '035',
       '037', '039', '041', '043', '045', '047', '049', '051', '053',
       '055', '057', '059', '061', '063', '065', '067', '069', '071',
       '073', '075', '077', '079', '081', '083', '085', '087', '089',
       '091', '093', '095', '097', '099', '101', '103', '105', '107',
       '109', '111', '113', '115', '117', '119', '121', '123', '125',
       '127', '129', '131', '133', '135', '137', '139', '141', '143',
       '145', '147', '149', '151', '153', '155', '157', '159', '161',
       '163', '165', '167', '169', '171', '173', '175', '177', '179',
       '181', '183', '185', '187', '189', '191', '193', '195', '197',
       '199', '201', '203', '205', '207', '209', '211', '213', '215',
       '217', '219', '221', '223', '225', '227', '229', '231', '233',
       '235', '237', '239', '241', '243', '245', '247', '249', '251',
       '253', '255',

In [ ]:
all_years_population_df["county"].unique()

array(['407', '459', '473', '493', '219', '329', '299', '147', '041',
       '165', '179', '237', '035', '029', '023', '121', '167', '393',
       '463', '037', '263', '425', '199', '289', '351', '389', '453',
       '087', '143', '345', '083', '325', '213', '277', '267', '477',
       '429', '003', '177', '305', '505', '221', '269', '073', '119',
       '253', '249', '111', '157', '235', '489', '115', '051', '355',
       '379', '385', '431', '065', '161', '197', '045', '291', '301',
       '395', '401', '455', '019', '075', '091', '225', '349', '415',
       '433', '021', '025', '103', '373', '391', '467', '055', '487',
       '317', '501', '189', '419', '387', '399', '435', '451', '469',
       '481', '485', '033', '315', '323', '181', '297', '363', '255',
       '123', '059', '067', '183', '313', '303', '257', '259', '153',
       '017', '071', '027', '061', '117', '141', '283', '443', '457',
       '483', '009', '043', '215', '423', '105', '207', '227', '333',
       '337', '365',

In [34]:
population_df = all_years_population_df

In [35]:
population_df['county'] = population_df['county'].astype(int)
merged_count_dist_mpo_1['county'] = merged_count_dist_mpo_1['county'].astype(int)

c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


#County population

In [36]:
merged_count_dist_mpo_pop = pd.merge(merged_count_dist_mpo_1, population_df, left_on='county', right_on='county', how='left')
merged_count_dist_mpo_pop.shape

(1016, 28)

In [37]:
merged_count_dist_mpo_pop.head()

,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state_x,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,...,Hispanic_or_Latino,Asian_Alone,Native_Hawaiian_or_Other_Pacific_Islander_Alone,Native_American_Alone,Other_Race,Two_or_More_Races,Some_Other_Race,Other_Race_Two_or_More,state_y,Year
0,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,19306.0,18083.0,82.0,208.0,181.0,266.0,43.0,1097.0,48,2019
1,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,19334.0,17064.0,63.0,281.0,241.0,222.0,57.0,2158.0,48,2020
2,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,18009.0,14788.0,191.0,359.0,382.0,292.0,26.0,3001.0,48,2021
3,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,17942.0,12954.0,194.0,347.0,401.0,507.0,31.0,4621.0,48,2022
4,48285,143,Lavaca,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",...,3785.0,1030.0,47.0,1563.0,125.0,115.0,5.0,2601.0,48,2019


Merging Monitored road to previously merged county, district and mpo to get the context of monitored roads

In [38]:
monitored_roads = monitored_roads.to_crs(merged_count_dist_mpo_1.crs)

# Perform spatial join (line segments within counties/districts)
monitored_roads_joined = gpd.sjoin(monitored_roads, merged_count_dist_mpo_1, how="inner", op="within")


c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [39]:
monitored_roads_joined.head()

,TOP100ID,Shape_Leng,geometry,index_right,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,MPO_NM,MPO_LBL,MPO_NBR
0,20010,0.070503,"LINESTRING (-10883707.611 3535522.977, -108836...",14,48385,193,Real,"POLYGON ((-11122837.374 3513019.751, -11120429...",48,385,SJT,7,San Angelo,"POLYGON ((-11244357.110 3774697.450, -11242321...",Capital Area,CAMPO,90.0
1,20021,0.061771,"LINESTRING (-10883028.270 3532242.024, -108830...",14,48385,193,Real,"POLYGON ((-11122837.374 3513019.751, -11120429...",48,385,SJT,7,San Angelo,"POLYGON ((-11244357.110 3774697.450, -11242321...",Capital Area,CAMPO,90.0
2,20040,0.043798,"LINESTRING (-10881200.799 3537558.993, -108812...",14,48385,193,Real,"POLYGON ((-11122837.374 3513019.751, -11120429...",48,385,SJT,7,San Angelo,"POLYGON ((-11244357.110 3774697.450, -11242321...",Capital Area,CAMPO,90.0
3,20041,0.063094,"LINESTRING (-10883640.489 3532711.750, -108836...",14,48385,193,Real,"POLYGON ((-11122837.374 3513019.751, -11120429...",48,385,SJT,7,San Angelo,"POLYGON ((-11244357.110 3774697.450, -11242321...",Capital Area,CAMPO,90.0
4,200370,0.076132,"LINESTRING (-11349531.655 4161886.255, -113493...",97,48239,121,Jackson,"POLYGON ((-10727433.778 3377232.647, -10726899...",48,239,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Amarillo,Amarillo MPO,120.0


In [40]:
monitored_roads.shape

(2310, 3)

In [41]:
merged_count_dist_mpo_pop.columns

Index(['FIPS_ST_CN', 'TXDOT_CNTY', 'CNTY_NM', 'geometry_x', 'state_x',
       'county', 'DIST_ABRVN', 'DIST_NBR', 'DIST_NM', 'geometry_y', 'MPO_NM',
       'MPO_LBL', 'MPO_NBR', 'geometry', 'Total_Population',
       'Not_Hispanic_or_Latino', 'White_Alone',
       'Black_or_African_American_Alone', 'Hispanic_or_Latino', 'Asian_Alone',
       'Native_Hawaiian_or_Other_Pacific_Islander_Alone',
       'Native_American_Alone', 'Other_Race', 'Two_or_More_Races',
       'Some_Other_Race', 'Other_Race_Two_or_More', 'state_y', 'Year'],
      dtype='object')

In [42]:
merged_count_dist_mpo_pop.shape

(1016, 28)

In [43]:
merged_count_dist_mpo_pop.head()

,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state_x,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,...,Hispanic_or_Latino,Asian_Alone,Native_Hawaiian_or_Other_Pacific_Islander_Alone,Native_American_Alone,Other_Race,Two_or_More_Races,Some_Other_Race,Other_Race_Two_or_More,state_y,Year
0,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,19306.0,18083.0,82.0,208.0,181.0,266.0,43.0,1097.0,48,2019
1,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,19334.0,17064.0,63.0,281.0,241.0,222.0,57.0,2158.0,48,2020
2,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,18009.0,14788.0,191.0,359.0,382.0,292.0,26.0,3001.0,48,2021
3,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,17942.0,12954.0,194.0,347.0,401.0,507.0,31.0,4621.0,48,2022
4,48285,143,Lavaca,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",...,3785.0,1030.0,47.0,1563.0,125.0,115.0,5.0,2601.0,48,2019


Urban areas percentage for each county/district

In [44]:
# Load the urban areas shapefile
urban_areas_us = gpd.read_file('urban_area/Urban_Area_20.shp')


In [45]:
county.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,geometry,state,county
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325


In [46]:
# Calculate the area for each county (in square meters by default in this CRS)
county['county_area'] = county['geometry'].area


In [47]:
county.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,geometry,state,county,county_area
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,5.353346e+09
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,3.324197e+09
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,5.068515e+09
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,3.738449e+09
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,4.583275e+09


In [2]:
#Adding land and water area
urban_areas_us['ubran_area'] = urban_areas_us['ALAND'] + urban_areas_us['AWATER']

NameError: name 'urban_areas_us' is not defined

In [49]:
urban_areas_us.head()

,GEOID,NAMELSAD,UATYP,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,ubran_area
0,00037,"Abbeville, LA Urban Area",U,28823510.0,340389.0,+29.9686495,-92.0952248,"MULTIPOLYGON (((-92.10126 29.96806, -92.10506 ...",29163899.0
1,00064,"Abbeville, SC Urban Area",U,12815034.0,6144.0,+34.1767288,-82.3829451,"POLYGON ((-82.35703 34.16871, -82.35724 34.168...",12821178.0
2,00145,"Aberdeen, SD Urban Area",U,35989073.0,183150.0,+45.4632038,-98.4688111,"MULTIPOLYGON (((-98.51086 45.48016, -98.51086 ...",36172223.0
3,00172,"Aberdeen, WA Urban Area",U,28416189.0,1802207.0,+46.9745945,-123.8318158,"POLYGON ((-123.81358 46.99439, -123.81359 46.9...",30218396.0
4,00253,"Abilene, KS Urban Area",U,9426821.0,1877.0,+38.9210604,-97.2207543,"POLYGON ((-97.21682 38.90533, -97.21667 38.905...",9428698.0


In [50]:
county = county.rename(columns={'geometry': 'county_geometry'})
county = county.set_geometry('county_geometry')
# Ensure both GeoDataFrames have the same CRS
urban_areas_us = urban_areas_us.to_crs(county.crs)

# Perform a spatial join to find urban areas within each Texas county
urban_areas_by_county = gpd.sjoin(urban_areas_us, county, how="inner", op="within")
urban_areas_by_county.head()

c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,GEOID,NAMELSAD,UATYP,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,ubran_area,index_right,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,state,county,county_area
5,00280,"Abilene, TX Urban Area",U,160509868.0,1015710.0,+32.4257902,-99.7498804,"MULTIPOLYGON (((-11096590.113 3819977.000, -11...",161525578.0,167,221,221,48441,221,8,Taylor,148,48,441,3.342056e+09
34,01414,"Alice, TX Urban Area",U,25980592.0,0.0,+27.7506221,-98.0734199,"MULTIPOLYGON (((-10922487.097 3215677.698, -10...",25980592.0,79,125,125,48249,126,16,Jim Wells,12,48,249,2.881682e+09
43,01711,"Alpine, TX Urban Area",U,11103460.0,688.0,+30.3602187,-103.6674137,"POLYGON ((-11540108.560 3548056.464, -11540111...",11104148.0,39,22,22,48043,22,24,Brewster,74,48,043,2.137658e+10
49,01900,"Alvarado, TX Urban Area",U,7883213.0,39327.0,+32.4122262,-97.2193506,"POLYGON ((-10821060.124 3817352.176, -10821068...",7922540.0,103,126,126,48251,127,2,Johnson,157,48,251,2.673476e+09
65,02467,"Andrews, TX Urban Area",U,24644900.0,30529.0,+32.3237431,-102.5411840,"POLYGON ((-11411812.958 3803954.600, -11411860...",24675429.0,101,2,2,48003,2,6,Andrews,155,48,003,5.458018e+09


In [51]:
urban_areas_by_county.columns

Index(['GEOID', 'NAMELSAD', 'UATYP', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry', 'ubran_area', 'index_right', 'CMPTRL_CNT', 'DPS_CNTY_N',
       'FIPS_ST_CN', 'TXDOT_CNTY', 'TXDOT_DIST', 'CNTY_NM', 'GID', 'state',
       'county', 'county_area'],
      dtype='object')

In [52]:
urban_areas_by_county = urban_areas_by_county[['TXDOT_CNTY', 'county_area','ubran_area',]]
urban_areas_by_county.head()

,TXDOT_CNTY,county_area,ubran_area
5,221,3.342056e+09,161525578.0
34,126,2.881682e+09,25980592.0
43,22,2.137658e+10,11104148.0
49,127,2.673476e+09,7922540.0
65,2,5.458018e+09,24675429.0


In [53]:
merged_urban_county = pd.merge(merged_count_dist_mpo_pop, urban_areas_by_county, left_on='TXDOT_CNTY', right_on='TXDOT_CNTY', how='left')
merged_urban_county.shape

(1128, 30)

In [55]:
merged_urban_county.tail()

,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state_x,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,...,Native_Hawaiian_or_Other_Pacific_Islander_Alone,Native_American_Alone,Other_Race,Two_or_More_Races,Some_Other_Race,Other_Race_Two_or_More,state_y,Year,county_area,ubran_area
1123,48237,120,Jack,"POLYGON ((-10900201.697 3953021.671, -10900201...",48,237,FTW,2,Fort Worth,"POLYGON ((-10906971.037 3957460.505, -10906975...",...,9.0,408.0,121.0,60.0,0.0,363.0,48,2022,NaN,NaN
1124,48127,64,Dimmit,"POLYGON ((-11144652.652 3330733.031, -11144652...",48,127,LRD,22,Laredo,"POLYGON ((-11250313.074 3540594.544, -11250313...",...,0.0,62.0,0.0,147.0,0.0,863.0,48,2019,4.431715e+09,11006796.0
1125,48127,64,Dimmit,"POLYGON ((-11144652.652 3330733.031, -11144652...",48,127,LRD,22,Laredo,"POLYGON ((-11250313.074 3540594.544, -11250313...",...,0.0,35.0,36.0,164.0,0.0,2839.0,48,2020,4.431715e+09,11006796.0
1126,48127,64,Dimmit,"POLYGON ((-11144652.652 3330733.031, -11144652...",48,127,LRD,22,Laredo,"POLYGON ((-11250313.074 3540594.544, -11250313...",...,0.0,44.0,39.0,140.0,0.0,3295.0,48,2021,4.431715e+09,11006796.0
1127,48127,64,Dimmit,"POLYGON ((-11144652.652 3330733.031, -11144652...",48,127,LRD,22,Laredo,"POLYGON ((-11250313.074 3540594.544, -11250313...",...,0.0,90.0,103.0,82.0,0.0,4596.0,48,2022,4.431715e+09,11006796.0


In [56]:
merged_urban_county["perc_urban"] = (merged_urban_county['ubran_area'] / merged_urban_county ['county_area'])*100


In [57]:
merged_urban_county.head()

,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state_x,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,...,Native_American_Alone,Other_Race,Two_or_More_Races,Some_Other_Race,Other_Race_Two_or_More,state_y,Year,county_area,ubran_area,perc_urban
0,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,208.0,181.0,266.0,43.0,1097.0,48,2019,5.353346e+09,19059512.0,0.35603
1,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,281.0,241.0,222.0,57.0,2158.0,48,2020,5.353346e+09,19059512.0,0.35603
2,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,359.0,382.0,292.0,26.0,3001.0,48,2021,5.353346e+09,19059512.0,0.35603
3,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",...,347.0,401.0,507.0,31.0,4621.0,48,2022,5.353346e+09,19059512.0,0.35603
4,48285,143,Lavaca,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",...,1563.0,125.0,115.0,5.0,2601.0,48,2019,NaN,NaN,NaN


#Land-Use within each county- Replica data

In [58]:
land_use = gpd.read_file('land-use_spring-2024_texas_shp/land-use_spring-2024_texas.shp')

In [59]:
land_use.head()

,geo_fips,name,la_total,la_single,la_multi,la_retail,la_office,la_attract,la_mu_res,la_mu_comm,...,la_industr,la_health,la_edu,la_civ_oth,la_transpo,la_space,la_agri,la_other,la_unknown,geometry
0,481677262001,"1 (Tract 7262, Galveston, TX)",391076867,18141276,4951919,6647415,458602,19010,738059,8142505,...,328261688,0,0,50034,5357851,3021735,5990253,728046,0,"POLYGON ((-94.95183 29.37916, -94.95151 29.379..."
1,482014516032,"2 (Tract 4516.03, Harris, TX)",3062837,1112227,693507,43942,101242,43597,535995,0,...,0,0,0,40145,0,0,0,0,0,"POLYGON ((-95.64458 29.75916, -95.64458 29.759..."
2,483030004091,"1 (Tract 4.09, Lubbock, TX)",4449144,3558621,66166,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-101.91387 33.60682, -101.91385 33.6..."
3,482014325011,"1 (Tract 4325.01, Harris, TX)",1312858,247268,519493,337154,0,208943,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-95.53516 29.72828, -95.53530 29.728..."
4,482015116003,"3 (Tract 5116, Harris, TX)",3354697,1019328,291020,289481,30081,161960,5010,0,...,402605,0,295176,48980,0,640372,0,102510,0,"POLYGON ((-95.38192 29.80371, -95.38192 29.803..."


In [ ]:
land_use['FIPS_ST_CN'] = land_use['geo_fips'].astype(str).str[:5]

In [63]:
land_use.head()

,geo_fips,name,la_total,la_single,la_multi,la_retail,la_office,la_attract,la_mu_res,la_mu_comm,...,la_health,la_edu,la_civ_oth,la_transpo,la_space,la_agri,la_other,la_unknown,geometry,FIPS_ST_CN
0,481677262001,"1 (Tract 7262, Galveston, TX)",391076867,18141276,4951919,6647415,458602,19010,738059,8142505,...,0,0,50034,5357851,3021735,5990253,728046,0,"POLYGON ((-94.95183 29.37916, -94.95151 29.379...",48167
1,482014516032,"2 (Tract 4516.03, Harris, TX)",3062837,1112227,693507,43942,101242,43597,535995,0,...,0,0,40145,0,0,0,0,0,"POLYGON ((-95.64458 29.75916, -95.64458 29.759...",48201
2,483030004091,"1 (Tract 4.09, Lubbock, TX)",4449144,3558621,66166,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-101.91387 33.60682, -101.91385 33.6...",48303
3,482014325011,"1 (Tract 4325.01, Harris, TX)",1312858,247268,519493,337154,0,208943,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-95.53516 29.72828, -95.53530 29.728...",48201
4,482015116003,"3 (Tract 5116, Harris, TX)",3354697,1019328,291020,289481,30081,161960,5010,0,...,0,295176,48980,0,640372,0,102510,0,"POLYGON ((-95.38192 29.80371, -95.38192 29.803...",48201


In [69]:
land_use.columns

Index(['geo_fips', 'name', 'la_total', 'la_single', 'la_multi', 'la_retail',
       'la_office', 'la_attract', 'la_mu_res', 'la_mu_comm', 'la_mu_indu',
       'la_mu_oth', 'la_industr', 'la_health', 'la_edu', 'la_civ_oth',
       'la_transpo', 'la_space', 'la_agri', 'la_other', 'la_unknown',
       'geometry', 'FIPS_ST_CN'],
      dtype='object')

In [ ]:
# Group by 'column_name' and apply multiple aggregations
grouped_land_use = land_use.groupby('FIPS_ST_CN').agg({
    'la_total': 'sum', 
    'la_single': 'sum', 
    'la_multi': 'sum', 
    'la_retail': 'sum',
    'la_office': 'sum', 
    'la_attract': 'sum', 
    'la_mu_res': 'sum', 
    'la_mu_comm': 'sum', 
    'la_mu_indu': 'sum',
    'la_mu_oth': 'sum', 
    'la_industr': 'sum', 
    'la_health': 'sum', 
    'la_edu': 'sum', 
    'la_civ_oth': 'sum',
    'la_transpo': 'sum', 
    'la_space': 'sum', 
    'la_agri': 'sum', 
    'la_other': 'sum', 
    'la_unknown': 'sum'
}).reset_index()


In [71]:
grouped_land_use.head()

,FIPS_ST_CN,la_total,la_single,la_multi,la_retail,la_office,la_attract,la_mu_res,la_mu_comm,la_mu_indu,la_mu_oth,la_industr,la_health,la_edu,la_civ_oth,la_transpo,la_space,la_agri,la_other,la_unknown
0,48001,29689793941,3472892668,34196956,184059826,8229997,22967159,26170659,19249614,9029688,12024826,72890431,222528,28234227,386190349,202210732,50110787,24744818675,416294819,0
1,48003,41539304938,400014797,46180778,13817933,6555961,9628375,609078,1793489,234517,1515435,17428971,902511,2204194,1257804,13383225,1120105,1826271,0,41020831494
2,48005,23663694053,4836976142,46825534,249010636,23818766,22633820,9763038,26841310,3649222,34144096,74489929,875507,59211936,38352419,45504753,2387124672,15607927010,193514610,3030653
3,48007,7533595836,420044583,27124571,34299472,3080462,3641942,3508075,7006291,261784,5816863,4040145,643949,5381516,2879934,26215657,4042557,0,0,6985608035
4,48009,25463353252,749834426,44857458,10598541,5072713,3471417,872786,2368237,201095,1408748,3151225,0,1330365,1197700,14318619,871209,23675759495,0,948039218


In [75]:
df_merged_land_use = county.merge(grouped_land_use, how='left', left_on='FIPS_ST_CN', right_on='FIPS_ST_CN')
df_merged_land_use.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,county_geometry,state,county,...,la_mu_oth,la_industr,la_health,la_edu,la_civ_oth,la_transpo,la_space,la_agri,la_other,la_unknown
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,...,6557896,34143244,913973,20623827,10125885,15598218,86207793,38999421626,0,1995810186
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,...,5944886,6752205,1761145,3351826,2551945,2502933,2226501,11223480,0,19954387864
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,...,787870166,583829743,1983489,82965609,403869965,194504134,7810042088,20092908495,499323347,0
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,...,3718209,27110523,721616,18245650,4896299,11222755,246857092,9881735570,0,16166290205
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,...,10989682,121040032,1016959,13283885,19280306,479445585,135499225,30173944789,431462059,0


#NOT REQUIRED ANYMORE

In [ ]:
# Load the urban areas shapefile
# previous file
land_use = gpd.read_file('Texas_landuse/land_use_filtered/Texas_landuse2 selection.shp')

In [49]:
land_use['landuse_area'] = land_use['geometry'].area
land_use.head()

C:\Users\S-Singh\AppData\Local\Temp\ipykernel_18588\4011484363.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  land_use['landuse_area'] = land_use['geometry'].area


,LU,county,geometry,landuse_area
0,residential,Austin,"POLYGON ((-97.78911 30.49815, -97.78915 30.498...",4.714926e-08
1,residential,Austin,"POLYGON ((-97.72000 30.29141, -97.72007 30.291...",4.310373e-08
2,vacant,Austin,"POLYGON ((-97.91230 30.25214, -97.91240 30.251...",1.221465e-05
3,residential,Austin,"POLYGON ((-97.87388 30.32685, -97.87402 30.326...",2.130390e-07
4,residential,Austin,"POLYGON ((-97.74100 30.22740, -97.74103 30.227...",1.401794e-08


In [50]:
land_use1 = land_use[["LU", "county", "landuse_area"]]

In [51]:
land_use_grouped = land_use1.groupby(['county','LU'])['landuse_area'].sum().reset_index()
land_use_grouped.head()

,county,LU,landuse_area
0,Austin,commercial,0.024830
1,Austin,residential,0.043314
2,Austin,vacant,0.073440
3,Bexar County,commercial,0.029495
4,Bexar County,industrial,0.003192


In [52]:
# Pivot table without 'date' as an index
land_use_wide = land_use_grouped.pivot_table(index='county', columns='LU', values='landuse_area', aggfunc='sum').reset_index()
land_use_wide.head()


LU,county,commercial,industrial,residential,vacant
0,Austin,0.024830,NaN,0.043314,0.073440
1,Bexar County,0.029495,0.003192,0.067961,0.165222
2,Bowie County,0.002013,0.000370,0.010052,0.212032
3,Brazos C,0.009989,0.001414,0.013442,0.100531
4,Cameron County,0.007347,0.000654,0.000009,0.137199


In [53]:
county['match_key'] = county['CNTY_NM']

# Perform the join using str.contains
df_merged = county.merge(land_use_wide, how='left', left_on='match_key', right_on=land_use_wide['county'].str.extract(r'(\w+)')[0])
df_merged.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,county_geometry,state,county_x,county_area,match_key,county_y,commercial,industrial,residential,vacant
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,5.353346e+09,Uvalde,NaN,NaN,NaN,NaN,NaN
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,3.324197e+09,Lavaca,NaN,NaN,NaN,NaN,NaN
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,5.068515e+09,Brazoria,NaN,NaN,NaN,NaN,NaN
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,3.738449e+09,Wharton,NaN,NaN,NaN,NaN,NaN
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,4.583275e+09,Medina,NaN,NaN,NaN,NaN,NaN


In [73]:
df_merged.shape

(254, 30)

In [ ]:
df_sorted = df_merged.sort_values(by='residential', ascending=False)
df_sorted.head()

In [56]:
land_use_wide.columns

Index(['county', 'commercial', 'industrial', 'residential', 'vacant'], dtype='object', name='LU')

In [57]:
land_use_grouped['county'].unique()

array(['Austin', 'Bexar County', 'Bowie County', 'Brazos C',
       'Cameron County', 'Collin County', 'Dallas C', 'Danton C',
       'Ector County', 'El Paso County', 'Fort Bend C', 'Galveston C',
       'Harris County', 'Hidalgo County', 'Lubbock County', 'McLennan C',
       'Midland C', 'Nueces County', 'Tarrant County', 'Washington C',
       'Webb County', 'Wichita County', 'Williamson C'], dtype=object)

In [58]:
county['CNTY_NM'].unique()

array(['Uvalde', 'Lavaca', 'Brazoria', 'Wharton', 'Medina', 'Galveston',
       'Bexar', 'Gonzales', 'Fort Bend', 'Guadalupe', 'Bandera',
       'Colorado', 'Chambers', 'Comal', 'Real', 'Caldwell', 'Kendall',
       'Austin', 'Fayette', 'Harris', 'Edwards', 'Kerr', 'Val Verde',
       'Waller', 'Jefferson', 'Hays', 'Orange', 'Bastrop', 'Washington',
       'Gillespie', 'Blanco', 'Lee', 'Liberty', 'Travis', 'Hardin',
       'Terrell', 'Montgomery', 'Sutton', 'Kimble', 'Brewster',
       'Presidio', 'Burleson', 'Williamson', 'Grimes', 'Llano', 'Mason',
       'San Jacinto', 'Brazos', 'Burnet', 'Menard', 'Schleicher',
       'Crockett', 'Walker', 'Milam', 'Madison', 'Jeff Davis', 'Tyler',
       'Polk', 'Jasper', 'Bell', 'Newton', 'Pecos', 'Robertson',
       'Lampasas', 'Trinity', 'San Saba', 'McCulloch', 'Irion', 'Cameron',
       'Willacy', 'Starr', 'Hidalgo', 'Brooks', 'Kenedy', 'Zapata',
       'Jim Hogg', 'Kleberg', 'Nueces', 'Duval', 'Jim Wells',
       'San Patricio', 'Webb', 'Ara

#UNTIL HERE

#Population raster data - 1,2,3 mile population around monitored roads

In [59]:
# Path to the population raster and the target CRS
population_raster_path = "landscan-usa-2021-day-assets/raster_clipped/landscan_clipped.tif"
output_raster_path = "landscan-usa-2021-day-assets/raster_clipped/reprojected_landscan_clipped.tif"
target_crs = "EPSG:32614"  # UTM Zone 14N (meters)

# Open the population raster
with rasterio.open(population_raster_path) as src:
    # Get the transform and dimensions for the new projected raster
    transform, width, height = calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds
    )

    # Update the metadata for the projected raster
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Open a new file to write the reprojected raster
    with rasterio.open(output_raster_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )

print(f"Raster reprojected and saved to {output_raster_path}")

Raster reprojected and saved to landscan-usa-2021-day-assets/raster_clipped/reprojected_landscan_clipped.tif


In [60]:
# Load the population raster
population_raster = rasterio.open("landscan-usa-2021-day-assets/raster_clipped/reprojected_landscan_clipped.tif")

In [61]:
raster_path = "landscan-usa-2021-day-assets/raster_clipped/reprojected_landscan_clipped.tif"

In [62]:
monitored_roads_projected = monitored_roads.to_crs(epsg=32614)

In [63]:
with rasterio.open(raster_path) as src:
    # Read the raster data
    image = src.read(1)  # Assuming the population data is in the first band
    mask = image != src.nodata  # Create a mask for non-nodata values
    
    # Extract the polygons and associated values
    results = (
        {'properties': {'value': v}, 'geometry': s}
        for s, v in shapes(image, mask=mask, transform=src.transform)
    )

# Convert to GeoDataFrame
geoms = list(results)
gdf = gpd.GeoDataFrame.from_features(geoms)

# Set the CRS to match the original raster
gdf.crs = src.crs


In [64]:
raster_pop = gdf

In [65]:
monitored_roads.head()

,TOP100ID,Shape_Leng,geometry
0,20010,0.070503,"LINESTRING (-10883707.611 3535522.977, -108836..."
1,20021,0.061771,"LINESTRING (-10883028.270 3532242.024, -108830..."
2,20040,0.043798,"LINESTRING (-10881200.799 3537558.993, -108812..."
3,20041,0.063094,"LINESTRING (-10883640.489 3532711.750, -108836..."
4,200370,0.076132,"LINESTRING (-11349531.655 4161886.255, -113493..."


In [66]:
raster_pop.head()

,geometry,value
0,"POLYGON ((-171612.561 4064928.615, -171612.561...",1.0
1,"POLYGON ((-166017.935 4064589.547, -166017.935...",1.0
2,"POLYGON ((-165933.168 4064589.547, -165933.168...",2.0
3,"POLYGON ((-165848.401 4064504.780, -165848.401...",1.0
4,"POLYGON ((-167713.276 4064250.479, -167713.276...",1.0


In [67]:
buffer_distances = {'1_mile': 1609.34, '2_mile': 1609.34 * 2, '3_mile': 1609.34 * 3}

# Create buffers and calculate population within each buffer
for distance_name, buffer_dist in buffer_distances.items():
    # Buffer around each line segment
    monitored_roads_projected[f'{distance_name}_buffer'] = monitored_roads_projected.geometry.buffer(buffer_dist)

    # Create a GeoDataFrame for the buffer
    buffer_gdf = gpd.GeoDataFrame(geometry=monitored_roads_projected[f'{distance_name}_buffer'], crs=monitored_roads_projected.crs)

    # Spatial join between buffered areas and population grid
    joined = gpd.sjoin(buffer_gdf, raster_pop, how='left', op='intersects')

    # Reset the index to avoid misalignment
    joined = joined.reset_index()

    # Sum the population within each buffer using the 'value' column
    population_sum = joined.groupby('index')['value'].sum()

    # Align the population sum to the original line segments' index
    monitored_roads_projected[f'{distance_name}_population'] = monitored_roads_projected.index.map(population_sum)

c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [68]:
monitored_roads_projected.head()

,TOP100ID,Shape_Leng,geometry,1_mile_buffer,1_mile_population,2_mile_buffer,2_mile_population,3_mile_buffer,3_mile_population
0,20010,0.070503,"LINESTRING (618337.712 3346953.543, 618384.753...","POLYGON ((618111.590 3345039.826, 618077.550 3...",71010.0,"POLYGON ((617436.698 3343575.889, 617223.606 3...",228357.0,"POLYGON ((616748.243 3342119.511, 616556.275 3...",381845.0
1,20021,0.061771,"LINESTRING (618955.554 3344137.888, 618949.606...","POLYGON ((615452.986 3339186.335, 615457.802 3...",51251.0,"POLYGON ((613965.533 3339859.268, 613980.971 3...",121348.0,"POLYGON ((612466.847 3340489.270, 612470.154 3...",202704.0
2,20040,0.043798,"LINESTRING (620485.159 3348728.059, 620466.688...","POLYGON ((617333.873 3345915.730, 617345.049 3...",186133.0,"POLYGON ((615532.294 3345950.073, 615531.193 3...",354801.0,"POLYGON ((614487.338 3347420.051, 614552.428 3...",466469.0
3,20041,0.063094,"LINESTRING (618421.884 3344536.218, 618394.437...","POLYGON ((614058.035 3339008.094, 614076.523 3...",47181.0,"POLYGON ((612491.330 3339378.034, 612508.854 3...",128813.0,"POLYGON ((610921.422 3339727.748, 610941.186 3...",229520.0
4,200370,0.076132,"LINESTRING (230300.498 3875404.189, 230442.685...","POLYGON ((230499.270 3877007.532, 230513.072 3...",15832.0,"POLYGON ((230423.257 3878620.542, 230422.928 3...",17364.0,"POLYGON ((228844.181 3880001.238, 228840.228 3...",18596.0


In [69]:
monitored_roads_projected[monitored_roads_projected["TOP100ID"] == 8000155]

,TOP100ID,Shape_Leng,geometry,1_mile_buffer,1_mile_population,2_mile_buffer,2_mile_population,3_mile_buffer,3_mile_population
1960,8000155,0.023314,"LINESTRING (704932.598 3629272.595, 704941.771...","POLYGON ((704248.611 3630727.455, 704274.993 3...",249625.0,"POLYGON ((703265.995 3632022.731, 703277.154 3...",351515.0,"POLYGON ((702248.039 3633280.895, 702269.966 3...",455809.0


#Crash data 2019,2021,2023 data

In [76]:
crash_data = pd.read_csv('crash_data/all_crash/Crash Location information 2019 2021 2023.csv')

In [77]:
crash_data.head()

,Crash_ID,Crash_Date,Intrsct_Relat_ID,Road_Part_Adj_ID,Road_Cls_ID,Road_Relat_ID,Cnty_ID,City_ID,Latitude,Longitude,...,Ref_Mark_Displ,Street_Name_2,Control_2,Section_2,Milepoint_2,Onsys_Fl,Rural_Fl,Crash_Sev_ID,Func_Sys_ID,Year
0,16359136,07/19/2023,2,1,5,1,15,1142,29.472914,-98.491692,...,NaN,AGNES DR,NaN,NaN,NaN,N,Y,5,NaN,2023
1,18118485,07/11/2023,2,1,5,2,182,294,32.825343,-98.110765,...,-0.536,NE 3RD AVE,NaN,NaN,NaN,Y,N,5,4.0,2023
2,18921290,06/01/2023,4,1,5,1,20,326,29.547332,-95.231940,...,NaN,NaN,NaN,NaN,NaN,N,N,5,NaN,2023
3,19280236,05/28/2023,4,1,5,2,61,710,33.058717,-97.123931,...,NaN,NaN,NaN,NaN,NaN,N,Y,5,NaN,2023
4,19379514,09/04/2023,4,1,2,2,99,352,34.297462,-99.739689,...,0.067,NaN,NaN,NaN,NaN,Y,Y,0,4.0,2023


In [78]:
crash_data['Cnty_ID'].unique().shape

(254,)

In [79]:
county.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,county_geometry,state,county,county_area
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,5.353346e+09
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,3.324197e+09
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,5.068515e+09
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,3.738449e+09
4,163,163,48325,163,15,Medina,39,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,4.583275e+09


In [80]:
# Create GeoDataFrame for crash points
gdf_crash_points = gpd.GeoDataFrame(crash_data, geometry=gpd.points_from_xy(crash_data['Longitude'], crash_data['Latitude']))

# Set the coordinate reference system (CRS) to match your data (e.g., EPSG:4326 for WGS84)
gdf_crash_points.set_crs(epsg=4326, inplace=True)

county = county.to_crs(gdf_crash_points.crs)
# Step 3: Perform a spatial join (join crash points with the counties)
county_crash = gpd.sjoin(gdf_crash_points, county, how="left", op='within')

c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [31]:
# Save the crash points GeoDataFrame as a shapefile
#gdf_crash_points.to_file('crash_data/crash_points.shp', driver='ESRI Shapefile')

C:\Users\S-Singh\AppData\Local\Temp\ipykernel_33376\103506153.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_crash_points.to_file('crash_data/crash_points.shp', driver='ESRI Shapefile')


In [81]:
county_crash.head()

,Crash_ID,Crash_Date,Intrsct_Relat_ID,Road_Part_Adj_ID,Road_Cls_ID,Road_Relat_ID,Cnty_ID,City_ID,Latitude,Longitude,...,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,state,county,county_area
0,16359136,07/19/2023,2,1,5,1,15,1142,29.472914,-98.491692,...,15.0,15.0,48029,15.0,15.0,Bexar,41.0,48,029,4.309667e+09
1,18118485,07/11/2023,2,1,5,2,182,294,32.825343,-98.110765,...,182.0,182.0,48363,182.0,2.0,Palo Pinto,174.0,48,363,3.618700e+09
2,18921290,06/01/2023,4,1,5,1,20,326,29.547332,-95.231940,...,20.0,20.0,48039,20.0,12.0,Brazoria,37.0,48,039,5.068515e+09
3,19280236,05/28/2023,4,1,5,2,61,710,33.058717,-97.123931,...,61.0,61.0,48121,61.0,18.0,Denton,192.0,48,121,3.530308e+09
4,19379514,09/04/2023,4,1,2,2,99,352,34.297462,-99.739689,...,99.0,99.0,48197,100.0,25.0,Hardeman,227.0,48,197,2.649490e+09


In [95]:

# Group by 'TXDOT_CNTY' and 'Year' and count the number of crashes
county_crash_counts = county_crash.groupby(['county', 'Year'])['geometry'].count().reset_index()

# Rename columns for clarity
county_crash_counts.columns = ['county', 'Year', 'Crash_Count']


In [96]:
county_crash_counts.shape

(762, 3)

In [98]:
county_crash_counts.tail(25)

,county,Year,Crash_Count
737,491,2023,9361
738,493,2019,556
739,493,2021,618
740,493,2023,600
741,495,2019,274
742,495,2021,166
743,495,2023,255
744,497,2019,928
745,497,2021,1018
746,497,2023,1145


Calculating crashes on monitored roads

In [99]:
monitored_roads.head()

,TOP100ID,Shape_Leng,geometry
0,20010,0.070503,"LINESTRING (-10883707.611 3535522.977, -108836..."
1,20021,0.061771,"LINESTRING (-10883028.270 3532242.024, -108830..."
2,20040,0.043798,"LINESTRING (-10881200.799 3537558.993, -108812..."
3,20041,0.063094,"LINESTRING (-10883640.489 3532711.750, -108836..."
4,200370,0.076132,"LINESTRING (-11349531.655 4161886.255, -113493..."


In [94]:
# Define the buffer distance (e.g., 10 meters)
buffer_distance = 10

# Ensure both GeoDataFrames have the same CRS
gdf_crash_points = gdf_crash_points.to_crs(monitored_roads.crs)

# Apply buffer to crash points
gdf_crash_points_buffered = gdf_crash_points.copy()
gdf_crash_points_buffered['geometry'] = gdf_crash_points.geometry.buffer(buffer_distance)

# Perform a spatial join with the buffered points
crash_monitored_roads = gpd.sjoin(gdf_crash_points_buffered, monitored_roads, how="inner", predicate="intersects")


In [99]:
crash_monitored_roads.head()

,Crash_ID,Crash_Date,Intrsct_Relat_ID,Road_Part_Adj_ID,Road_Cls_ID,Road_Relat_ID,Cnty_ID,City_ID,Latitude,Longitude,...,Milepoint_2,Onsys_Fl,Rural_Fl,Crash_Sev_ID,Func_Sys_ID,Year,geometry,index_right,TOP100ID,Shape_Leng
2,18921290,06/01/2023,4,1,5,1,20,326,29.547332,-95.231940,...,NaN,N,N,5,NaN,2023,"POLYGON ((-10601161.088 3445495.347, -10601161...",499,1200252,0.118607
6,19463928,05/15/2023,2,1,5,1,101,208,29.787395,-95.617683,...,NaN,N,N,5,NaN,2023,"POLYGON ((-10644101.785 3476250.597, -10644101...",549,1200299,0.124977
7,19467960,04/01/2023,3,2,2,1,57,218,32.837200,-96.976140,...,NaN,Y,N,3,2.0,2023,"POLYGON ((-10795324.491 3873714.904, -10795324...",1882,8000078,0.102096
8,19467995,04/01/2023,3,1,5,1,15,379,29.549115,-98.374232,...,NaN,N,N,5,NaN,2023,"POLYGON ((-10950959.451 3445723.442, -10950959...",1071,1900079,0.095199
11,19468066,04/01/2023,3,1,2,1,212,434,32.295073,-95.261136,...,NaN,Y,N,5,3.0,2023,"POLYGON ((-10604411.154 3802106.182, -10604411...",1279,2200054,0.099362


In [100]:
# Group by 'TXDOT_CNTY' and 'Year' and count the number of crashes
monitored_road_crash_counts = crash_monitored_roads.groupby(['TOP100ID', 'Year'])['geometry'].count().reset_index()

# Rename columns for clarity
monitored_road_crash_counts.columns = ['TOP100ID', 'Year', 'Crash_Count']

In [101]:
monitored_road_crash_counts.head()

,TOP100ID,Year,Crash_Count
0,20010,2019,172
1,20010,2021,128
2,20010,2023,114
3,20021,2019,199
4,20021,2021,191


#Non attainment area

In [78]:
shapefile_path7= "non_attainment/nonattainment_and_maintenance_areas_-7471405565837912373/2008_and_2015_Nonattinment.shp"
non_attainment = gpd.read_file(shapefile_path7)

In [79]:
non_attainment.head()

,NAAQS_1997,NAAQS_2008,NAAQS_2015,Attainment,Nonattainm,CNTY_NM,geometry
0,Attainment Maintenance,nonattainment,nonattainment,1997,2008/2015,Denton,"POLYGON ((-97.38279 33.43034, -97.38191 33.430..."
1,Attainment Maintenance,nonattainment,nonattainment,1997,2008/2015,Collin,"POLYGON ((-96.83387 33.40520, -96.83294 33.405..."
2,Attainment Maintenance,nonattainment,None,1997,2008,Rockwall,"POLYGON ((-96.51659 32.98220, -96.51186 32.982..."
3,Attainment Maintenance,nonattainment,nonattainment,1997,2008/2015,Dallas,"POLYGON ((-96.84441 32.98744, -96.84227 32.987..."
4,Attainment Maintenance,nonattainment,nonattainment,1997,2008/2015,Tarrant,"POLYGON ((-97.54387 32.99405, -97.54313 32.994..."


In [64]:
non_attainment.shape

(22, 7)

In [80]:
county_non_attainment = county

In [81]:
county_non_attainment["non_attainment"] = np.where(
    county_non_attainment['CNTY_NM'].isin(non_attainment['CNTY_NM']), 
    1, 
    0
)

In [72]:
county_non_attainment.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,geometry,non_attainment
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-99.50031 29.08711, -99.50147 29.087...",0
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-96.86553 29.63014, -96.86540 29.630...",0
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-95.28413 29.59773, -95.28403 29.597...",1
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-96.17527 29.63380, -96.17501 29.633...",0
4,163,163,48325,163,15,Medina,39,"POLYGON ((-99.41315 29.17354, -99.41315 29.180...",0


#weather data

In [82]:
shapefile_path10 = "noaa_weather_texas/storm_events_texas_2010.shp"
weather = gpd.read_file(shapefile_path10)

In [83]:
weather.head()

,OBJECTID,State,Month,Event_Type,Begin_Date,Property_D,Begin_Lat,Begin_Lon,Episode_Na,Event_Narr,...,Direct_Dea,Episode_ID,Source,Data_Sourc,Tornado_F_,County_Z_2,Total_Inju,x,y,geometry
0,372493,TEXAS,December,Frost/Freeze,26-DEC-10 05:00:00,0.00K,None,None,"A developing cold front, associated with the c...",The cooperative observer at Raymondville repor...,...,0,45890,COOP Observer,CSV,None,WILLACY,0,-97.648174,26.471159,POINT (-97.64817 26.47116)
1,372494,TEXAS,December,Frost/Freeze,26-DEC-10 06:00:00,0.00K,None,None,"A developing cold front, associated with the c...",The coldest recorded temperatures of the 2010/...,...,0,45890,ASOS,CSV,None,CAMERON,0,-97.524242,26.135977,POINT (-97.52424 26.13598)
2,372526,TEXAS,December,Frost/Freeze,13-DEC-10 02:00:00,0.00K,None,None,A fairly vigorous cool front swept through Dee...,Freezing temperatures reached the Zapata Count...,...,0,45889,COOP Observer,CSV,None,ZAPATA,0,-99.167027,27.009983,POINT (-99.16703 27.00998)
3,372531,TEXAS,November,High Wind,01-NOV-10 01:51:00,0.00K,None,None,Increasing westerly flow aloft due to a passin...,Sustained winds of 40 mph or greater funneled ...,...,0,45560,AWOS,CSV,None,GUADALUPE MOUNTAINS OF CULBERSON COUNTY,0,-104.860496,31.891228,POINT (-104.86050 31.89123)
4,372532,TEXAS,November,High Wind,15-NOV-10 15:40:00,0.00K,None,None,Isolated strong wind gusts occurred in the hig...,High winds occurred in the higher elevations o...,...,0,45561,Other Federal Agency,CSV,None,DAVIS / APACHE MOUNTAINS AREA,0,-104.356873,31.185974,POINT (-104.35687 31.18597)


In [86]:
county.head()

,CMPTRL_CNT,DPS_CNTY_N,FIPS_ST_CN,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,GID,county_geometry,state,county,county_area,match_key,non_attainment
0,232,232,48463,232,15,Uvalde,35,"POLYGON ((-99.50031 29.08711, -99.50147 29.087...",48,463,5.353346e+09,Uvalde,0
1,143,143,48285,143,13,Lavaca,36,"POLYGON ((-96.86553 29.63014, -96.86540 29.630...",48,285,3.324197e+09,Lavaca,0
2,20,20,48039,20,12,Brazoria,37,"POLYGON ((-95.28413 29.59773, -95.28403 29.597...",48,039,5.068515e+09,Brazoria,1
3,241,241,48481,241,13,Wharton,38,"POLYGON ((-96.17527 29.63380, -96.17501 29.633...",48,481,3.738449e+09,Wharton,0
4,163,163,48325,163,15,Medina,39,"POLYGON ((-99.41315 29.17354, -99.41315 29.180...",48,325,4.583275e+09,Medina,0


In [87]:
county_for_weather = county[['TXDOT_CNTY', 'TXDOT_DIST','CNTY_NM', 'county', 'county_geometry']]

In [88]:
weather = weather.to_crs(county_for_weather.crs)

# Perform the spatial join - joining points to the county polygons
# This will assign the county each point falls within
weather_county = gpd.sjoin(weather, county_for_weather, how="left", op="within")

c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [89]:
weather_county.head()

,OBJECTID,State,Month,Event_Type,Begin_Date,Property_D,Begin_Lat,Begin_Lon,Episode_Na,Event_Narr,...,County_Z_2,Total_Inju,x,y,geometry,index_right,TXDOT_CNTY,TXDOT_DIST,CNTY_NM,county
0,372493,TEXAS,December,Frost/Freeze,26-DEC-10 05:00:00,0.00K,None,None,"A developing cold front, associated with the c...",The cooperative observer at Raymondville repor...,...,WILLACY,0,-97.648174,26.471159,POINT (-97.64817 26.47116),69.0,245.0,21.0,Willacy,489
1,372494,TEXAS,December,Frost/Freeze,26-DEC-10 06:00:00,0.00K,None,None,"A developing cold front, associated with the c...",The coldest recorded temperatures of the 2010/...,...,CAMERON,0,-97.524242,26.135977,POINT (-97.52424 26.13598),68.0,31.0,21.0,Cameron,061
2,372526,TEXAS,December,Frost/Freeze,13-DEC-10 02:00:00,0.00K,None,None,A fairly vigorous cool front swept through Dee...,Freezing temperatures reached the Zapata Count...,...,ZAPATA,0,-99.167027,27.009983,POINT (-99.16703 27.00998),74.0,253.0,21.0,Zapata,505
3,372531,TEXAS,November,High Wind,01-NOV-10 01:51:00,0.00K,None,None,Increasing westerly flow aloft due to a passin...,Sustained winds of 40 mph or greater funneled ...,...,GUADALUPE MOUNTAINS OF CULBERSON COUNTY,0,-104.860496,31.891228,POINT (-104.86050 31.89123),147.0,55.0,24.0,Culberson,109
4,372532,TEXAS,November,High Wind,15-NOV-10 15:40:00,0.00K,None,None,Isolated strong wind gusts occurred in the hig...,High winds occurred in the higher elevations o...,...,DAVIS / APACHE MOUNTAINS AREA,0,-104.356873,31.185974,POINT (-104.35687 31.18597),147.0,55.0,24.0,Culberson,109


TXDOT projects shapefile

In [23]:
merged_count_dist_mpo_1.head()

,FIPS_ST_CN,TXDOT_CNTY,CNTY_NM,geometry_x,state,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,MPO_NM,MPO_LBL,MPO_NBR,geometry
0,48463,232,Uvalde,"POLYGON ((-11076323.621 3386738.492, -11076452...",48,463,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",South East Texas Regional Planning Commission,South East Texas RPC MPO,135.0,"POLYGON ((-10472027.847 3571354.051, -10472022..."
1,48285,143,Lavaca,"POLYGON ((-10783021.183 3456095.210, -10783006...",48,285,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Victoria,Victoria MPO,363.0,"POLYGON ((-10792171.645 3384911.927, -10791169..."
2,48039,20,Brazoria,"POLYGON ((-10606980.495 3451945.262, -10606969...",48,039,HOU,12,Houston,"POLYGON ((-10642059.507 3569163.765, -10642010...",Bryan-College Station,Bryan-College Station MPO,249.0,"POLYGON ((-10713061.325 3627506.843, -10713050..."
3,48481,241,Wharton,"POLYGON ((-10706181.772 3456563.942, -10706152...",48,481,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Texarkana,Texarkana MPO,211.0,"POLYGON ((-10486938.781 3949087.997, -10487007..."
4,48325,163,Medina,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",Waco,Waco MPO,140.0,"POLYGON ((-10833157.277 3666435.715, -10836210..."


In [7]:
shapefile_path10 = "TXDOT_Projects/TXDOT_Projects.shp"
TXDOT_projects = gpd.read_file(shapefile_path10)

In [8]:
TXDOT_projects.head()

,OBJECTID,CONTROL_SE,DISTRICT_N,COUNTY_NUM,HIGHWAY_NU,PROJ_CLASS,TYPE_OF_WO,LIMITS_FRO,LIMITS_TO,BEG_MILE_P,...,LET_TYPE_D,MPO_NM,CSJ_TEXT,NHS_FLAG,PROJECT_ID,PROJ_STG,PROJ_STAT,ESTMTD_FIS,SHAPE_Leng,geometry
0,1,023202020,9.0,74.0,SH 53,Overlay,Preventive Maintenance,BELL CO LINE,US 77,0.986,...,Statewide Let,None,None,N,A00004626,Planning,Active,2030,0.154982,"LINESTRING Z (-97.12908 31.07047 0.00000, -97...."
1,2,059601020,9.0,110.0,FM 66,Bridge Replacement,REPL BR & APPRS,@ WEIR BRANCH,(STR #012),14.375,...,Statewide Let,NA,None,N,A00002047,PS&E,Active,2024,0.000062,"LINESTRING Z (-97.07449 32.24106 0.00000, -97...."
2,4,004903069,9.0,74.0,SH 6,Seal Coat,PREVENTATIVE MAINTENANCE,FM 2307,BIG CREEK,5.948,...,Statewide Let,None,None,Y,A00004826,Construction,Active,2020,0.133179,"LINESTRING Z (-96.90341 31.37629 0.00000, -96...."
3,5,206001038,9.0,161.0,FM 2113,Safety Improvement Projects,Widen Road - Add Shoulders,FM 2416,FM 107,8.098,...,Statewide Let,Waco,None,N,A00004847,Planning,Active,2025,0.120778,"LINESTRING Z (-97.29043 31.39951 0.00000, -97...."
4,6,023201053,9.0,14.0,SH 53,Bicycle Infrastructure Improvements,Construct Pedestrian Infrastructure,SPUR 290,MARTIN LUTHER KING JR. BLVD,0.000,...,Local Agency Let,Killeen-Temple,None,Y,A00004699,Construction,Active,2021,0.005828,"LINESTRING Z (-97.34270 31.09819 0.00000, -97...."


In [24]:
TXDOT_projects = TXDOT_projects.to_crs(merged_count_dist_mpo_1.crs)

# Perform spatial join (line segments within counties/districts)
TXDOT_projects_joined = gpd.sjoin(TXDOT_projects, merged_count_dist_mpo_1, how="inner", op="within")


c:\Users\S-Singh\AppData\Local\R-MINI~1\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [26]:
TXDOT_projects_joined.columns

Index(['OBJECTID', 'CONTROL_SE', 'DISTRICT_N', 'COUNTY_NUM', 'HIGHWAY_NU',
       'PROJ_CLASS', 'TYPE_OF_WO', 'LIMITS_FRO', 'LIMITS_TO', 'BEG_MILE_P',
       'END_MILE_P', 'DIST_LET_D', 'TRUNK_SYS_', 'FREIGHT', 'ENERGY',
       'PROJ_LENGT', 'RAIL_COORD', 'ACTUAL_LET', 'CONTROL__1', 'DISTRICT_1',
       'COUNTY_NAM', 'PT_TOP100', 'PT_PHASE', 'TPP_WORK_P', 'PRJ_TIER',
       'PRJ_UTP', 'PRJ_UTP_Tx', 'STATE_HSE_', 'US_HSE_NBR', 'STATE_SEN_',
       'PROJ_ESTMT', 'COMMISSION', 'EST_CONSTR', 'LET_TYPE_C', 'LET_TYPE_D',
       'MPO_NM_left', 'CSJ_TEXT', 'NHS_FLAG', 'PROJECT_ID', 'PROJ_STG',
       'PROJ_STAT', 'ESTMTD_FIS', 'SHAPE_Leng', 'geometry', 'index_right',
       'FIPS_ST_CN', 'TXDOT_CNTY', 'CNTY_NM', 'geometry_x', 'state', 'county',
       'DIST_ABRVN', 'DIST_NBR', 'DIST_NM', 'geometry_y', 'MPO_NM_right',
       'MPO_LBL', 'MPO_NBR'],
      dtype='object')

In [25]:
TXDOT_projects_joined.head()

,OBJECTID,CONTROL_SE,DISTRICT_N,COUNTY_NUM,HIGHWAY_NU,PROJ_CLASS,TYPE_OF_WO,LIMITS_FRO,LIMITS_TO,BEG_MILE_P,...,geometry_x,state,county,DIST_ABRVN,DIST_NBR,DIST_NM,geometry_y,MPO_NM_right,MPO_LBL,MPO_NBR
3,5,206001038,9.0,161.0,FM 2113,Safety Improvement Projects,Widen Road - Add Shoulders,FM 2416,FM 107,8.098,...,"POLYGON ((-11066621.012 3397753.016, -11066621...",48,325,SAT,15,San Antonio,"POLYGON ((-11103341.987 3540952.941, -11103526...",Waco,Waco MPO,140.0
4,6,023201053,9.0,14.0,SH 53,Bicycle Infrastructure Improvements,Construct Pedestrian Infrastructure,SPUR 290,MARTIN LUTHER KING JR. BLVD,0.000,...,"POLYGON ((-10740733.126 3486568.725, -10740668...",48,089,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Killeen-Temple,Killeen-Temple MPO,277.0
5,7,032002039,9.0,14.0,SH 95,Bridge Replacement,Bridge Replacement,@ WILLOW CREEK,(STR #049),2.871,...,"POLYGON ((-10740733.126 3486568.725, -10740668...",48,089,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Killeen-Temple,Killeen-Temple MPO,277.0
6,8,032006008,9.0,14.0,SL 363,Widen Non-Freeway,Construct New Road,INDUSTRIAL BLVD,LUCIUS MCCELVEY,1.336,...,"POLYGON ((-10740733.126 3486568.725, -10740668...",48,089,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Killeen-Temple,Killeen-Temple MPO,277.0
7,9,213701020,9.0,14.0,FM 2271,Intersection & Operational Imprv,Construct New Road,0.146 MI N OF FM 439,FM 439,1.805,...,"POLYGON ((-10740733.126 3486568.725, -10740668...",48,089,YKM,13,Yoakum,"POLYGON ((-10758694.531 3522214.151, -10758867...",Killeen-Temple,Killeen-Temple MPO,277.0
